In [1]:
import pandas as pd
import pickle
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Load cleaned heart dataset
csv_path = Path("c:/4TH SEM/Group Project/Code/data/cleaned_heart.csv")
df = pd.read_csv(csv_path)

# One-hot encode categorical features to match app expectations
cat_cols = ["gender", "cholesterol", "gluc"]
df = pd.get_dummies(df, columns=cat_cols, prefix=cat_cols, drop_first=True)

# Ensure dummy columns exist in fixed order
expected_dummies = ["gender_2", "cholesterol_2", "cholesterol_3", "gluc_2", "gluc_3"]
for col in expected_dummies:
    if col not in df:
        df[col] = 0

# Final feature order aligned with the Streamlit app and scaler
feature_cols = [
    "id",
    "age",
    "height",
    "weight",
    "systolic_bp",
    "diastolic_bp",
    "smoke",
    "alco",
    "active",
    "bmi",
    "gender_2",
    "cholesterol_2",
    "cholesterol_3",
    "gluc_2",
    "gluc_3",
]

X = df[feature_cols]
y = df["target"]
print(f"Encoded feature matrix: {X.shape}")

Encoded feature matrix: (68889, 15)


In [2]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
print(X_train.shape, X_test.shape)

(55111, 15) (13778, 15)


In [3]:
# Scale features and train Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(
    max_iter=3000,
    solver="liblinear",
    C=0.5,
    class_weight="balanced",
)
model.fit(X_train_scaled, y_train)

LogisticRegression(C=0.5, class_weight='balanced', max_iter=3000,
                   solver='liblinear')

In [4]:
# Evaluation
y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print("Heart Disease Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Heart Disease Accuracy: 0.7278269705327334
              precision    recall  f1-score   support

           0       0.71      0.78      0.74      6959
           1       0.75      0.68      0.71      6819

    accuracy                           0.73     13778
   macro avg       0.73      0.73      0.73     13778
weighted avg       0.73      0.73      0.73     13778



In [5]:
# Save scaler and model aligned with encoded features
models_dir = Path("c:/4TH SEM/Group Project/Code/models")
models_dir.mkdir(parents=True, exist_ok=True)

with open(models_dir / "heart_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

with open(models_dir / "heart_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Heart scaler saved")
print("✅ Heart Logistic Regression model saved")

✅ Heart scaler saved
✅ Heart Logistic Regression model saved
